<a href="https://colab.research.google.com/github/rath3t/maninColab/blob/main/compareInterpolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython --upgrade

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,181 kB]
Hit:13 http:

     |████████████████████████████████| 793 kB 15.7 MB/s 
     |████████████████████████████████| 381 kB 48.7 MB/s 
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: IPython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.29 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.33.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.


In [1]:
from manim import *

Manim Community v0.15.2

In [8]:
%%manim -qm  CompareUnitSphereInterpolation

import numpy
from manim import *


class CompareUnitSphereInterpolation(MovingCameraScene):
    def construct(self):
        rad = 2
        self.camera.background_color = WHITE

        circle = Circle(radius=rad, color=BLUE)
        t1_vec = circle.get_center() + rad * UP + 3 * LEFT * rad
        t2_vec = circle.get_center() + rad * UP + 2 * RIGHT * rad

        t1_vec = t1_vec / np.linalg.norm(t1_vec) * rad
        t2_vec = t2_vec / np.linalg.norm(t2_vec) * rad

        t1_vec0 = t1_vec.copy()

        t1_dot = Dot(color=BLACK).move_to(t1_vec)
        t2_dot = Dot(color=BLACK).move_to(t2_vec)
        t_nfe = t1_dot.copy()
        tnfeV = ValueTracker(0)

        def nfe_interpolate(t):
            return t1_dot.get_center() + (t2_dot.get_center() - t1_dot.get_center()) * t

        f_always(t_nfe.move_to, lambda: nfe_interpolate(tnfeV.get_value()))

        def pbfe_interpolate(t):
            vec = t1_dot.get_center() + (t2_dot.get_center() - t1_dot.get_center()) * t
            return vec / np.linalg.norm(vec) * rad

        def gfe_interpolate(t):
            omega = numpy.arccos(numpy.dot(t1_dot.get_center() / np.linalg.norm(t1_dot.get_center()),
                                           t2_dot.get_center() / np.linalg.norm(t2_dot.get_center())))
            vec = numpy.sin((1 - t) * omega) / numpy.sin(omega) * t1_dot.get_center() + numpy.sin(t * omega) / numpy.sin(
                omega) * t2_dot.get_center()
            return vec

        tpbfeV = ValueTracker(0)

        t_pbfe = t1_dot.copy().set_color(RED)
        f_always(t_pbfe.move_to, lambda: pbfe_interpolate(tpbfeV.get_value()))

        tgfeV = ValueTracker(0)

        t_gfe = t1_dot.copy().set_color(BLUE)
        f_always(t_gfe.move_to, lambda: gfe_interpolate(tgfeV.get_value()))

        arrow_t1 = Arrow(start=ORIGIN, end=t1_vec, color=GREEN)
        arrow_t2 = Arrow(start=ORIGIN, end=t2_vec, color=GREEN)

        line_t1_t2_nfe = Line(t1_dot.get_center(), t2_dot.get_center(), color=BLACK)
        line_t1_t2_nfe.add_updater(lambda m: line_t1_t2_nfe.become(Line(t1_dot.get_center(), t2_dot.get_center(), color=BLACK)))

        self.play(Create(circle))
        self.play(Create(Dot(color=BLACK).move_to(circle.get_center())))

        self.play(Create(t1_dot))
        self.play(Create(t2_dot))

        self.play(Create(line_t1_t2_nfe))

        self.play(Create(t_nfe))

        text, number = label = VGroup(
            Tex(r'$\xi= $').set_color(BLACK),
            DecimalNumber(0, show_ellipsis=True, num_decimal_places=2, include_sign=False, unit=None,color=BLACK).move_to(3*RIGHT)
        )
        label.arrange(RIGHT)
        label.move_to(2.5*UP)
        self.play(Create(label))
        label.add_updater(lambda d: number.set_value(tnfeV.get_value()))

        self.play(tnfeV.animate.set_value(1), run_time=3)
        self.play(tnfeV.animate.set_value(0.6), run_time=3)

        self.play(Create(t_pbfe))
        label.add_updater(lambda d: number.set_value(tpbfeV.get_value()))
        self.play(tpbfeV.animate.set_value(1), run_time=3)
        self.play(tpbfeV.animate.set_value(0.6), run_time=3)

        self.play(Create(t_gfe))
        label.add_updater(lambda d: number.set_value(tgfeV.get_value()))
        self.play(tgfeV.animate.set_value(1), run_time=3)
        self.play(tgfeV.animate.set_value(0.6), run_time=3)

        angletracer = ValueTracker(0)
        zoomedSet = VGroup(t_gfe,t1_dot,t2_dot,t_nfe,t_pbfe)

        f_always(t1_dot.move_to, lambda: rotate_vector((t1_vec0 - circle.get_center()), angletracer.get_value()))
        self.play(angletracer.animate.set_value(-0.5), run_time=3)
        self.play(self.camera.auto_zoom(zoomedSet), run_time=3)
        self.play(angletracer.animate.set_value(-1.5), run_time=3)

        self.camera.frame.save_state()
        self.wait(2)


[05/11/22 12:00:08] INFO     Animation 0 : Partial movie file        ]8;id=375191;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=318359;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py#506\506]8;;\
                             written in '/content/media/videos/conte                         
                             nt/720p30/partial_movie_files/CompareUn                         
                             itSphereInterpolation/3324555078_637955                         
                             927_223132457.mp4'                                              

[05/11/22 12:00:09] INFO     Animation 1 : Partial movie file        ]8;id=407616;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=134649;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py#506\506]8;;\
                             written in '/content/media/videos/conte                         
                             nt/720p30/partial_movie_files/CompareUn                         
                             itSphereInterpolation/2920019931_179400                         
                             8678_2548375731.mp4'                                            

[05/11/22 12:00:10] INFO     Animation 2 : Partial movie file        ]8;id=810691;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=438158;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py#506\506]8;;\
                             written in '/content/media/videos/conte                         
                             nt/720p30/partial_movie_files/CompareUn                         
                             itSphereInterpolation/2920019931_314010                         
                             7768_272548092.mp4'                                             

[05/11/22 12:00:11] INFO     Animation 3 : Partial movie file        ]8;id=767722;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=494936;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py#506\506]8;;\
                             written in '/content/media/videos/conte                         
                             nt/720p30/partial_movie_files/CompareUn                         
                             itSphereInterpolation/2920019931_279867                         
                             771_2859424299.mp4'                                             

[05/11/22 12:00:12] INFO     Animation 4 : Partial movie file        ]8;id=18933;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=852367;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py#506\506]8;;\
                             written in '/content/media/videos/conte                         
                             nt/720p30/partial_movie_files/CompareUn                         
                             itSphereInterpolation/2920019931_224325                         
                             3663_631511023.mp4'                                             

...

[05/11/22 12:01:07] INFO     Animation 17 : Partial movie file       ]8;id=135515;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=487683;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py#506\506]8;;\
                             written in '/content/media/videos/conte                         
                             nt/720p30/partial_movie_files/CompareUn                         
                             itSphereInterpolation/3138694827_416702                         
                             801_2706238997.mp4'                                             

[05/11/22 12:01:08] INFO     Animation 18 : Partial movie file       ]8;id=178035;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=942958;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py#506\506]8;;\
                             written in '/content/media/videos/conte                         
                             nt/720p30/partial_movie_files/CompareUn                         
                             itSphereInterpolation/3045161735_221171                         
                             4945_2617488730.mp4'                                            

                    INFO     Combining to Movie file.                ]8;id=518291;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=915229;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py#597\597]8;;\

                    INFO                                             ]8;id=232476;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=368212;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py#718\718]8;;\
                             File ready at '/content/media/videos/co                         
                             ntent/720p30/CompareUnitSphereInterpola                         
                             tion.mp4'                                                       
                                                                                             

                    INFO     Rendered CompareUnitSphereInterpolation             ]8;id=10539;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=8012;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene.py#241\241]8;;\
                             Played 19 animations                                            